<a href="https://colab.research.google.com/github/fullstackdata/public2024/blob/main/milvusbm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install -U pymilvus
# !pip install -U transformers accelerate bitsandbytes
# !pip install  llama-index-vector-stores-milvus llama-index-llms-huggingface llama-index-embeddings-huggingface sentence-transformers einops
# !pip install -U llama-index-vector-stores-milvus wikipedia-api
# !pip install -U FlagEmbedding
# !pip install -U datasets fsspec
!pip install -U sqlite4


In [1]:
from pymilvus import MilvusClient
import os

# Try a new unique name for the database file
db_file_name = "milvus_demo_new3.db"

# Optional: Clean up if it exists from a previous failed run
if os.path.exists(db_file_name):
    os.remove(db_file_name)
    print(f"Removed existing {db_file_name}")

try:
    client = MilvusClient(db_file_name)
    print(f"Successfully started Milvus Lite with database: {db_file_name}")

    # You can now proceed with your Milvus operations
    # Example:
    # client.create_collection(
    #     collection_name="my_collection",
    #     dimension=128,
    #     metric_type="L2",
    #     # Add a primary field and vector field if needed
    # )
    # print("Collection created.")

except Exception as e:
    print(f"Error starting Milvus Lite: {e}")
    # You can print more detailed traceback if needed:
    # import traceback
    # traceback.print_exc()

ERROR:root:Start milvus-lite failed
2025-06-25 13:01:16,357 [ERROR][_create_connection]: Failed to create new connection using: 9fa22253208e4ad7a09a6aec9176ac37 (milvus_client.py:916)


Error starting Milvus Lite: <ConnectionConfigException: (code=1, message=Open local milvus failed)>


In [ ]:
# Create an index over the documents
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.vector_stores.milvus.utils import BM25BuiltInFunction
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.vector_stores.milvus import MilvusVectorStore

bm25_function = BM25BuiltInFunction(
    analyzer_params={
        "tokenizer": "standard",
        "filter": [
            "lowercase",  # Built-in filter
            {"type": "length", "max": 40},  # Custom cap size of a single token
            {"type": "stop", "stop_words": ["of", "to"]},  # Custom stopwords
        ],
    },
    enable_match=True,
)


from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model_name = "microsoft/phi-4"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto' )

from llama_index.core.llms import ChatMessage
from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(tokenizer=tokenizer, model=model, device_map='auto')

from llama_index.core import Settings
Settings.llm = llm

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True, device='cuda')
Settings.embed_model = embed_model

In [ ]:
# # prompt: use hugging face datasets to download argilla/news dataset and save it as jsonl file

# # !pip install datasets
# from datasets import load_dataset
# import json

# # Load the dataset
# dataset = load_dataset("argilla/news")

# # Save the dataset to a JSONL file
# with open("argilla_news.jsonl", "w") as f:
#   for item in dataset["train"]:
#     f.write(json.dumps(item) + "\n")

# print("Dataset saved to argilla_news.jsonl")

In [ ]:
from llama_index.core import Document
import json

documents = []
with open('/content/argilla_news.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

for itm in data:
  documents.append(Document(text=itm["text"]))


from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.milvus.utils import BGEM3SparseEmbeddingFunction

# # bm25_function needs to be defined or imported if used
# # Assuming bm25_function is defined elsewhere or is a placeholder
# bm25_function = None # Replace with actual BM25 function if needed

fields = [
    {"name": "embedding", "type": "FLOAT_VECTOR", "dim": 768}, # Assuming your dense embeddings have dimension 768
    {"name": "text", "type": "VARCHAR", "max_length": 65535},
    {"name": "sparse_embedding", "type": "SPARSE_FLOAT_VECTOR"}, # Add the sparse embedding field
]

vector_store = MilvusVectorStore(
    "./milvus_llamaindex.db",
    dim=768,
    # enable_sparse=True,
    # sparse_embedding_function=BGEM3SparseEmbeddingFunction(),
    # fields=fields,
    # overwrite=True,
    # drop_old=True,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, llm=llm)

from llama_index.core.prompts import RichPromptTemplate
from llama_index.core import PromptTemplate

template = """
You are a Q&A chat bot.
Use the given context only, answer the question.
If you don't know the answer, say that you don't know the answer.

<context>
{context_str}
</context>

Question: {query_str}
"""

prompt_template = PromptTemplate(template=template)
qe = index.as_query_engine(embed_model=embed_model, similarity_top_k=10)
qe.update_prompts({"response_synthesizer:text_qa_template": prompt_template})




In [ ]:
response = qe.query("What are some of the films mentioned?")
print(response)



In [ ]:
for nd in response.source_nodes:
  print("***************")
  print(nd.node.text)